In [ ]:
df = pd.read_csv('/Users/landsiedelj/Downloads/hundertausend_csv.csv')

In [ ]:
%%time
def createText(x):
    try:
        import pandas as pd
        dfcontent = pd.json_normalize(x.content, max_level=1)
        dfcontent = dfcontent[dfcontent.subtype == 'paragraph']
        #dfcontent = dfcontent[dfcontent.mime == 'text/html']
        contentText = ''
        
        for index, row in dfcontent.iterrows():
            contentText += row["content"]
            contentText += ' '
            
            import re
            contentText = re.sub('<[^<]+?>', '', contentText)
            contentText = contentText.replace(u'\xa0', u' ')
        #return dfcontent
        return contentText
    except:
        return None

# converts the epochs datetime to a normale datestring
df['text'] = df.parallel_apply(createText, axis=1)

# if content json is converted in text delete the column
# !!! df = df.drop(columns='content')

In [ ]:
def cleaning_for_NER(df):
    
    dictionary = {
              "post_id"                : "id",
              "text"                : "article_text"
                }
    
    df = df.copy()
    df.rename(columns=dictionary, inplace=True)  # rename columns
    df.drop_duplicates(subset=['id'], inplace=True)

    # Remove "\n" in article_text
    #df["content"] = df["content"].apply(lambda x: re.sub("\{'content': '"," ", str(x)))
 
    return df    

In [ ]:
# Safe cleaned data in new df
df = cleaning_for_NER(df)

In [ ]:
# drop empty rows
nan_value = float("NaN")
#df.replace("", 'no title', inplace=True)
df["title"].replace(np.NaN, '', inplace=True)
df["article_text"].replace(np.NaN, '', inplace=True)

#df.dropna(subset = ["title"], inplace=True)

In [ ]:
# Combine column "title" & "content" into "merged_total_text":
df['merged_total_text'] = df.apply(lambda r: r['title'] + '. '+ r['article_text'], axis=1)

In [ ]:
del df['article_text']
del df['content']

In [ ]:
df.to_json('/Users/landsiedelj/Downloads/hundredthousand_wp_merged_text.jsonl', lines=True, orient='records')